# Data

In [15]:
import pandas as pd


# Read the CSV file into a DataFrame
eeg1 = pd.read_csv("EEG Data/Copy of fused_transposed_GG1.csv")
emg1 = pd.read_csv("EMG Data/Copy of fused_transposed_GG1.csv")
comb = pd.read_csv("Combined Data/Copy of combined_features.csv")
# Display the first few rows
# print(comb.head())
comb

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Class
0,7493.390318,7.136456,7493.390318,478.029614,3.181524e+10,7.136456,478.502127,2.436395e+07,3.617252e+08,6.696118e+11,0.0
1,7465.559631,6.832010,7465.559631,477.358359,3.178214e+10,6.832010,473.332925,2.373346e+07,3.518449e+08,6.688907e+11,0.0
2,7451.721034,6.587196,7451.721034,488.144035,3.177560e+10,6.587196,462.174017,2.245591e+07,3.319692e+08,6.687391e+11,0.0
3,7437.794622,5.590710,7437.794622,503.786732,3.177446e+10,5.590710,448.899346,2.110148e+07,3.112887e+08,6.686976e+11,0.0
4,7397.362225,4.144632,7397.362225,519.579525,3.176317e+10,4.144632,433.346133,1.952560e+07,2.870129e+08,6.684369e+11,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3869,10020.546160,-2.341174,10020.546160,177.731824,4.447729e+10,-2.341174,151.995658,1.574423e+06,1.861456e+07,9.580765e+11,3.0
3870,10019.879267,-1.355867,10019.879267,182.641861,4.445148e+10,-1.355867,147.643357,1.483075e+06,1.747371e+07,9.574985e+11,3.0
3871,9996.794041,-0.674178,9996.794041,184.311748,4.441715e+10,-0.674178,144.749170,1.426587e+06,1.676490e+07,9.567003e+11,3.0
3872,10002.519461,0.926101,10002.519461,185.866086,4.441556e+10,0.926101,142.995837,1.414205e+06,1.663390e+07,9.566782e+11,3.0


In [16]:
comb.head().shape

(5, 11)

In [17]:
# Define the window size
window_size = 10

# Group by 'Class' and split into windows
windows = []
for class_label, group in comb.groupby('Class'):
    for i in range(0, len(group), window_size):
        window = group.iloc[i:i + window_size]
        if len(window) == window_size:
            windows.append(window)

# View the resulting windows
for idx, window in enumerate(windows):
    print(f"Window {idx + 1}:\n{window}\n")

Window 1:
      Feature1  Feature2     Feature3    Feature4      Feature5  Feature6  \
0  7493.390318  7.136456  7493.390318  478.029614  3.181524e+10  7.136456   
1  7465.559631  6.832010  7465.559631  477.358359  3.178214e+10  6.832010   
2  7451.721034  6.587196  7451.721034  488.144035  3.177560e+10  6.587196   
3  7437.794622  5.590710  7437.794622  503.786732  3.177446e+10  5.590710   
4  7397.362225  4.144632  7397.362225  519.579525  3.176317e+10  4.144632   
5  7374.590652  2.815403  7374.590652  530.659072  3.177713e+10  2.815403   
6  7413.905017  0.220081  7413.905017  536.331210  3.181923e+10  0.220081   
7  7432.024102 -1.792994  7432.024102  528.937445  3.184961e+10 -1.792994   
8  7415.094903 -4.060554  7415.094903  518.329333  3.185454e+10 -4.060554   
9  7407.358459 -5.554027  7407.358459  503.169834  3.185764e+10 -5.554027   

     Feature7      Feature8      Feature9     Feature10  Class  
0  478.502127  2.436395e+07  3.617252e+08  6.696118e+11    0.0  
1  473.33292

In [18]:
import pandas as pd
import numpy as np
import torch

# Initialize lists for features and targets
features = []
targets = []

# Loop through each window
for window in windows:
    features.append(window.iloc[:, :-1].values)  # All columns except the last
    targets.append(window.iloc[:, -1].values)   # Last column (Class)

# Convert lists to numpy arrays for further processing
features = np.array(features)  # Shape: (num_windows, 10, num_features)
targets = np.array(targets)    # Shape: (num_windows, 10)

# Optional: Check the shapes
print("Features shape:", features.shape)
print("Targets shape:", targets.shape)

# Convert to PyTorch tensors
features_tensor = torch.tensor(features, dtype=torch.float32)
target_tensor = torch.tensor(target, dtype=torch.long)  # Use long for classification labels

# Print shapes to verify
print(f"Features tensor shape: {features_tensor.shape}")
print(f"Target tensor shape: {target_tensor.shape}")


Features shape: (387, 10, 10)
Targets shape: (387, 10)
Features tensor shape: torch.Size([387, 10, 10])
Target tensor shape: torch.Size([3874])


In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming `features` and `targets` are NumPy arrays
# Convert features and targets to PyTorch tensors
features_tensor = torch.tensor(features, dtype=torch.float32)  # Convert to float tensor
target_tensor = torch.tensor(targets, dtype=torch.long)        # Convert to long tensor for classification tasks

# Create a dataset
dataset = TensorDataset(features_tensor, target_tensor)

# Define batch size
batch_size = 32

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example: Iterate through the DataLoader
for batch_features, batch_labels in dataloader:
    print("Batch features shape:", batch_features.shape)
    print("Batch labels shape:", batch_labels.shape)



Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size([32, 10, 10])
Batch labels shape: torch.Size([32, 10])
Batch features shape: torch.Size

# Model

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from functools import partial
from dualvit import DualViT, dualvit_s
from timm.models.vision_transformer import _cfg


In [21]:
# Define the updated dualvit_mod_s
def dualvit_mod_s(pretrained=False, **kwargs):
    model = DualViT(
        stem_hidden_dim=32,  # Can adjust if necessary
        embed_dims=[64, 128, 320, 448],  # Match the feature dimensions to your input
        num_heads=[2, 4, 10, 14],
        mlp_ratios=[8, 8, 4, 3],
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        depths=[3, 4, 6, 3],  # Layers in each block
        input_size=10,  # Input sequence length
        num_classes=10,  # Number of classes (per timestep)
        **kwargs,
    )
    model.default_cfg = _cfg()
    return model

def simulate_image(batch_size=1, in_chans=3, height=224, width=224):
    return torch.rand(batch_size, in_chans, height, width)

# Parameters
num_classes = 4
batch_size = 16
num_epochs = 5
learning_rate = 0.001

# Instantiate model, loss function, and optimizer
model = dualvit_mod_s()
model.train()  # Set to training mode
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Fake data loader
def fake_data_loader(num_batches, batch_size, in_chans, height, width, num_classes):
    for _ in range(num_batches):
        images = simulate_image(batch_size, in_chans, height, width)
        labels = torch.randint(0, num_classes, (batch_size,))
        yield images, labels

# Training loop
print("Starting Training...")
total_loss = 0
for images, labels in fake_data_loader(10, batch_size, 3, 224, 224, num_classes):
    optimizer.zero_grad()  # Clear gradients
    outputs = model(images)  # Forward pass
    
    # Extract logits if the model returns a tuple
    if isinstance(outputs, tuple):
        logits = outputs[0]
    else:
        logits = outputs

    loss = criterion(logits, labels)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights
    total_loss += loss.item()


# Testing loop
print("\nStarting Testing...")
model.eval()  # Set to evaluation mode
num_correct = 0
num_samples = 0
with torch.no_grad():
    for images, labels in fake_data_loader(5, batch_size, 3, 224, 224, num_classes):
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Get class with highest probability
        num_correct += (predictions == labels).sum().item()
        num_samples += labels.size(0)

accuracy = num_correct / num_samples * 100
print(f"Testing Accuracy: {accuracy:.2f}%")


Starting Training...

Starting Testing...
Testing Accuracy: 20.00%
